## 1. Cài đặt thư viện

In [1]:
import pandas as pd
import numpy as np
import math as mt

## 2. Đọc file dữ liệu

In [2]:
data_df = pd.read_csv('data/netflix_titles.csv')
data_df.head(3)
data_df.shape
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6234 entries, 0 to 6233
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       6234 non-null   int64 
 1   type          6234 non-null   object
 2   title         6234 non-null   object
 3   director      4265 non-null   object
 4   cast          5664 non-null   object
 5   country       5758 non-null   object
 6   date_added    6223 non-null   object
 7   release_year  6234 non-null   int64 
 8   rating        6224 non-null   object
 9   duration      6234 non-null   object
 10  listed_in     6234 non-null   object
 11  description   6234 non-null   object
dtypes: int64(2), object(10)
memory usage: 584.6+ KB


# 3. Tiền xử lý dữ liệu

## Xóa một số Features: show_id, release_year, description, director, cast	
### Chuyển date_added về year_added và month_added


In [3]:
del data_df['show_id']
del data_df['release_year']
del data_df['description']
del data_df['director']
del data_df['cast']
data_df["date_added"] = pd.to_datetime(data_df['date_added'])
data_df['year_added'] = data_df['date_added'].dt.year
data_df['month_added'] = data_df['date_added'].dt.month
del data_df['date_added']

## Xử lý missing data
### Các features: director, cast, country, date_added, rating chứa missing values.

Các trường: date_added và rating chứa missing data không nhiều, xóa đi các giá trị missing đó

In [4]:
data = data_df.dropna(subset=['year_added', 'rating'])

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6214 entries, 0 to 6222
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   type         6214 non-null   object 
 1   title        6214 non-null   object 
 2   country      5743 non-null   object 
 3   rating       6214 non-null   object 
 4   duration     6214 non-null   object 
 5   listed_in    6214 non-null   object 
 6   year_added   6214 non-null   float64
 7   month_added  6214 non-null   float64
dtypes: float64(2), object(6)
memory usage: 436.9+ KB


## Chỉnh sửa date_added theo year và month

In [6]:
data[0:5]

,type,title,country,rating,duration,listed_in,year_added,month_added
0,Movie,Norm of the North: King Sized Adventure,"United States, India, South Korea, China",TV-PG,90 min,"Children & Family Movies, Comedies",2019.0,9.0
1,Movie,Jandino: Whatever it Takes,United Kingdom,TV-MA,94 min,Stand-Up Comedy,2016.0,9.0
2,TV Show,Transformers Prime,United States,TV-Y7-FV,1 Season,Kids' TV,2018.0,9.0
3,TV Show,Transformers: Robots in Disguise,United States,TV-Y7,1 Season,Kids' TV,2018.0,9.0
4,Movie,#realityhigh,United States,TV-14,99 min,Comedies,2017.0,9.0


### Moive = 1, TV Show = 0

In [7]:
data['type'] = data['type'].map({'Movie': 1, 'TV Show': 0})

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Country

In [8]:

data_country = (data['country'].dropna()).values
country = []
for i in range(len(data_country)):
    country = country + data_country[i].split(", ")
country = np.unique(np.array(country),return_counts= True)
print("count_country:", len(country[0]))
#print(country[0])
name_country = country[0]
count_country = country[1]
dict_country = dict()
for i in range(len(country[0])):
    dict_country[name_country[i]] = count_country[i]
print(dict_country)

count_country: 113
{'Afghanistan': 1, 'Albania': 1, 'Argentina': 68, 'Armenia': 1, 'Australia': 124, 'Austria': 10, 'Azerbaijan': 1, 'Bangladesh': 3, 'Belgium': 66, 'Bermuda': 1, 'Botswana': 1, 'Brazil': 66, 'Bulgaria': 8, 'Cambodia': 3, 'Canada': 318, 'Cayman Islands': 1, 'Chile': 23, 'China': 120, 'Colombia': 36, 'Croatia': 4, 'Cuba': 1, 'Cyprus': 1, 'Czech Republic': 15, 'Denmark': 38, 'Dominican Republic': 2, 'East Germany': 1, 'Ecuador': 1, 'Egypt': 54, 'Finland': 9, 'France': 271, 'Georgia': 2, 'Germany': 151, 'Ghana': 2, 'Greece': 8, 'Guatemala': 1, 'Hong Kong': 97, 'Hungary': 5, 'Iceland': 6, 'India': 838, 'Indonesia': 52, 'Iran': 4, 'Iraq': 2, 'Ireland': 33, 'Israel': 26, 'Italy': 66, 'Japan': 229, 'Jordan': 6, 'Kazakhstan': 1, 'Kenya': 1, 'Kuwait': 1, 'Latvia': 1, 'Lebanon': 9, 'Liechtenstein': 1, 'Luxembourg': 7, 'Malawi': 1, 'Malaysia': 21, 'Malta': 2, 'Mauritius': 1, 'Mexico': 129, 'Mongolia': 1, 'Montenegro': 1, 'Morocco': 6, 'Namibia': 1, 'Nepal': 2, 'Netherlands': 37, '

In [9]:
mean = np.mean(count_country)
mean

63.39823008849557

In [10]:
data["country"].fillna('Brazil', inplace = True)

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6214 entries, 0 to 6222
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   type         6214 non-null   int64  
 1   title        6214 non-null   object 
 2   country      6214 non-null   object 
 3   rating       6214 non-null   object 
 4   duration     6214 non-null   object 
 5   listed_in    6214 non-null   object 
 6   year_added   6214 non-null   float64
 7   month_added  6214 non-null   float64
dtypes: float64(2), int64(1), object(5)
memory usage: 436.9+ KB


In [12]:
data_country = data['country'].values
country = []
for i in range(len(data_country)):
    country = country + data_country[i].split(", ")
country = np.unique(np.array(country),return_counts= True)
print("count_country:", len(country[0]))
#print(country[0])
name_country = country[0]
count_country = country[1]
dict_country = dict()
for i in range(len(country[0])):
    dict_country[name_country[i]] = count_country[i]
print(dict_country)

count_country: 113
{'Afghanistan': 1, 'Albania': 1, 'Argentina': 68, 'Armenia': 1, 'Australia': 124, 'Austria': 10, 'Azerbaijan': 1, 'Bangladesh': 3, 'Belgium': 66, 'Bermuda': 1, 'Botswana': 1, 'Brazil': 537, 'Bulgaria': 8, 'Cambodia': 3, 'Canada': 318, 'Cayman Islands': 1, 'Chile': 23, 'China': 120, 'Colombia': 36, 'Croatia': 4, 'Cuba': 1, 'Cyprus': 1, 'Czech Republic': 15, 'Denmark': 38, 'Dominican Republic': 2, 'East Germany': 1, 'Ecuador': 1, 'Egypt': 54, 'Finland': 9, 'France': 271, 'Georgia': 2, 'Germany': 151, 'Ghana': 2, 'Greece': 8, 'Guatemala': 1, 'Hong Kong': 97, 'Hungary': 5, 'Iceland': 6, 'India': 838, 'Indonesia': 52, 'Iran': 4, 'Iraq': 2, 'Ireland': 33, 'Israel': 26, 'Italy': 66, 'Japan': 229, 'Jordan': 6, 'Kazakhstan': 1, 'Kenya': 1, 'Kuwait': 1, 'Latvia': 1, 'Lebanon': 9, 'Liechtenstein': 1, 'Luxembourg': 7, 'Malawi': 1, 'Malaysia': 21, 'Malta': 2, 'Mauritius': 1, 'Mexico': 129, 'Mongolia': 1, 'Montenegro': 1, 'Morocco': 6, 'Namibia': 1, 'Nepal': 2, 'Netherlands': 37, 

In [13]:
a = [data_country[i].split(", ")[0] for i in range(len(data_country))]

In [14]:
a

['United States',
 'United Kingdom',
 'United States',
 'United States',
 'United States',
 'Spain',
 'Bulgaria',
 'Chile',
 'United States',
 'United States',
 'Brazil',
 'Netherlands',
 'Brazil',
 'Brazil',
 'Brazil',
 'Brazil',
 'Brazil',
 'Brazil',
 'Brazil',
 'France',
 'United States',
 'France',
 'United States',
 'United States',
 'United States',
 'United States',
 'Brazil',
 'United States,',
 'Brazil',
 'Thailand',
 'China',
 'United States',
 'Belgium',
 'Belgium',
 'Brazil',
 'India',
 'Brazil',
 'India',
 'Brazil',
 'France',
 'United States',
 'India',
 'United Kingdom',
 'United Kingdom',
 'Brazil',
 'United States',
 'Thailand',
 'Thailand',
 'Thailand',
 'Thailand',
 'Thailand',
 'Thailand',
 'United States',
 'United States',
 'United States',
 'Pakistan',
 'Canada',
 'United States',
 'India',
 'United States',
 'United Kingdom',
 'United States',
 'United Kingdom',
 'United States',
 'United States',
 'United States',
 'United States',
 'United States',
 'United St

In [15]:
print(len(a))

6214


In [16]:
data["country_film"] = a

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
del data["country"]
data['country_film'] = data['country_film'].map(dict_country)


c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
data

,type,title,rating,duration,listed_in,year_added,month_added,country_film
0,1,Norm of the North: King Sized Adventure,TV-PG,90 min,"Children & Family Movies, Comedies",2019.0,9.0,2600
1,1,Jandino: Whatever it Takes,TV-MA,94 min,Stand-Up Comedy,2016.0,9.0,600
2,0,Transformers Prime,TV-Y7-FV,1 Season,Kids' TV,2018.0,9.0,2600
3,0,Transformers: Robots in Disguise,TV-Y7,1 Season,Kids' TV,2018.0,9.0,2600
4,1,#realityhigh,TV-14,99 min,Comedies,2017.0,9.0,2600
...,...,...,...,...,...,...,...,...
6218,0,Talking Tom and Friends,TV-G,2 Seasons,"Kids' TV, TV Comedies",2019.0,4.0,1
6219,0,Pokémon the Series,TV-Y7-FV,2 Seasons,"Anime Series, Kids' TV",2019.0,4.0,229
6220,0,Justin Time,TV-Y,2 Seasons,Kids' TV,2016.0,4.0,318
6221,0,Terrace House: Boys & Girls in the City,TV-14,2 Seasons,"International TV Shows, Reality TV",2016.0,4.0,229


### Title

### listed_in

In [19]:
data_listed_in = data['listed_in'].values
listed_in = []
for i in range(len(data_listed_in)):
    listed_in= listed_in + data_listed_in[i].split(", ")
listed_in= np.unique(np.array(listed_in),return_counts= True)
name_listed_in = listed_in[0]
count_listed_in = listed_in[1]
dict_listed_in = dict()
for i in range(len(listed_in[0])):
    dict_listed_in[name_listed_in[i]] = count_listed_in[i]
print(dict_listed_in)
b = [data_listed_in [i].split(", ")[0] for i in range(len(data_listed_in))]
data["data_listed_in"] = b
del data["listed_in"]
data['data_listed_in'] = data['data_listed_in'].map(dict_listed_in )

{'Action & Adventure': 597, 'Anime Features': 45, 'Anime Series': 115, 'British TV Shows': 209, 'Children & Family Movies': 378, 'Classic & Cult TV': 22, 'Classic Movies': 84, 'Comedies': 1113, 'Crime TV Shows': 362, 'Cult Movies': 55, 'Documentaries': 668, 'Docuseries': 278, 'Dramas': 1622, 'Faith & Spirituality': 47, 'Horror Movies': 262, 'Independent Movies': 552, 'International Movies': 1927, 'International TV Shows': 999, "Kids' TV": 325, 'Korean TV Shows': 132, 'LGBTQ Movies': 60, 'Movies': 49, 'Music & Musicals': 243, 'Reality TV': 153, 'Romantic Movies': 376, 'Romantic TV Shows': 278, 'Sci-Fi & Fantasy': 193, 'Science & Nature TV': 67, 'Spanish-Language TV Shows': 116, 'Sports Movies': 157, 'Stand-Up Comedy': 281, 'Stand-Up Comedy & Talk Shows': 42, 'TV Action & Adventure': 125, 'TV Comedies': 428, 'TV Dramas': 598, 'TV Horror': 54, 'TV Mysteries': 69, 'TV Sci-Fi & Fantasy': 67, 'TV Shows': 10, 'TV Thrillers': 44, 'Teen TV Shows': 44, 'Thrillers': 392}


c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [20]:
data

,type,title,rating,duration,year_added,month_added,country_film,data_listed_in
0,1,Norm of the North: King Sized Adventure,TV-PG,90 min,2019.0,9.0,2600,378
1,1,Jandino: Whatever it Takes,TV-MA,94 min,2016.0,9.0,600,281
2,0,Transformers Prime,TV-Y7-FV,1 Season,2018.0,9.0,2600,325
3,0,Transformers: Robots in Disguise,TV-Y7,1 Season,2018.0,9.0,2600,325
4,1,#realityhigh,TV-14,99 min,2017.0,9.0,2600,1113
...,...,...,...,...,...,...,...,...
6218,0,Talking Tom and Friends,TV-G,2 Seasons,2019.0,4.0,1,325
6219,0,Pokémon the Series,TV-Y7-FV,2 Seasons,2019.0,4.0,229,115
6220,0,Justin Time,TV-Y,2 Seasons,2016.0,4.0,318,325
6221,0,Terrace House: Boys & Girls in the City,TV-14,2 Seasons,2016.0,4.0,229,999


### rating

In [21]:
data_rating = data['rating'].values
rating = []
for i in range(len(data_rating)):
    rating= rating + data_rating[i].split(", ")
rating= np.unique(np.array(rating),return_counts= True)
name_rating = rating[0]
count_rating = rating[1]
dict_rating = dict()
for i in range(len(rating[0])):
    dict_rating[name_rating[i]] = count_rating[i]
print(dict_rating)
data['data_rating'] = data_rating
data['data_rating'] = data['data_rating'].map(dict_rating)
del data['rating']

{'G': 37, 'NC-17': 2, 'NR': 217, 'PG': 184, 'PG-13': 286, 'R': 508, 'TV-14': 1695, 'TV-G': 149, 'TV-MA': 2025, 'TV-PG': 699, 'TV-Y': 142, 'TV-Y7': 168, 'TV-Y7-FV': 95, 'UR': 7}


c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [22]:
data

,type,title,duration,year_added,month_added,country_film,data_listed_in,data_rating
0,1,Norm of the North: King Sized Adventure,90 min,2019.0,9.0,2600,378,699
1,1,Jandino: Whatever it Takes,94 min,2016.0,9.0,600,281,2025
2,0,Transformers Prime,1 Season,2018.0,9.0,2600,325,95
3,0,Transformers: Robots in Disguise,1 Season,2018.0,9.0,2600,325,168
4,1,#realityhigh,99 min,2017.0,9.0,2600,1113,1695
...,...,...,...,...,...,...,...,...
6218,0,Talking Tom and Friends,2 Seasons,2019.0,4.0,1,325,149
6219,0,Pokémon the Series,2 Seasons,2019.0,4.0,229,115,95
6220,0,Justin Time,2 Seasons,2016.0,4.0,318,325,142
6221,0,Terrace House: Boys & Girls in the City,2 Seasons,2016.0,4.0,229,999,1695


In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6214 entries, 0 to 6222
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   type            6214 non-null   int64  
 1   title           6214 non-null   object 
 2   duration        6214 non-null   object 
 3   year_added      6214 non-null   float64
 4   month_added     6214 non-null   float64
 5   country_film    6214 non-null   int64  
 6   data_listed_in  6214 non-null   int64  
 7   data_rating     6214 non-null   int64  
dtypes: float64(2), int64(4), object(2)
memory usage: 436.9+ KB


### duration

In [24]:
dict_minitue = {'1 Season': 240,  '2 Seasons': 480, '3 Seasons': 720, '4 Seasons': 960,
 '5 Seasons': 1500,
 '6 Seasons': 1500,
 '7 Seasons': 1500,
 '8 Seasons': 1500,
 '9 Seasons': 1500,
 '10 Seasons': 1500,
 '11 Seasons': 1500,
 '12 Seasons': 1500,
 '13 Seasons': 1500,
 '14 Seasons': 1500,
 '15 Seasons': 1500}

In [25]:
data['duration'].replace({'min': ' '}, inplace=True, regex=True)
data['duration'].replace(dict_minitue, inplace=True, regex=True)
data["duration"] = pd.to_numeric(data["duration"])
data.info()

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\pandas\core\generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6214 entries, 0 to 6222
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   type            6214 non-null   int64  
 1   title           6214 non-null   object 
 2   duration        6214 non-null   int64  
 3   year_added      6214 non-null   float64
 4   month_added     6214 non-null   float64
 5   country_film    6214 non-null   int64  
 6   data_listed_in  6214 non-null   int64  
 7   data_rating     6214 non-null   int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 436.9+ KB


c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [26]:
data

,type,title,duration,year_added,month_added,country_film,data_listed_in,data_rating
0,1,Norm of the North: King Sized Adventure,90,2019.0,9.0,2600,378,699
1,1,Jandino: Whatever it Takes,94,2016.0,9.0,600,281,2025
2,0,Transformers Prime,240,2018.0,9.0,2600,325,95
3,0,Transformers: Robots in Disguise,240,2018.0,9.0,2600,325,168
4,1,#realityhigh,99,2017.0,9.0,2600,1113,1695
...,...,...,...,...,...,...,...,...
6218,0,Talking Tom and Friends,480,2019.0,4.0,1,325,149
6219,0,Pokémon the Series,480,2019.0,4.0,229,115,95
6220,0,Justin Time,480,2016.0,4.0,318,325,142
6221,0,Terrace House: Boys & Girls in the City,480,2016.0,4.0,229,999,1695
